In [36]:
import math
import geopandas as gpd
from shapely.geometry import Point, Polygon
import pandas as pd
import numpy as np
from fiona.drvsupport import supported_drivers
from simplekml import Kml, Color
supported_drivers['KML'] = 'rw'
supported_drivers['libkml'] = 'rw' # enable KML support which is disabled by default
supported_drivers['LIBKML'] = 'rw' # enable KML support which is disabled 
import numpy as np
import geopandas as gpd
from simplekml import Kml, Color
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import ListedColormap
from matplotlib import cm
import matplotlib.colors as colors
import matplotlib.colors as mcolors
import os
import time
from geopy.distance import geodesic

In [39]:
def generate_bins(geom, bin_size=10):

    x_ref, y_ref = 193602, 2602072
    xmin, ymin, xmax, ymax = geom.bounds
    # To shift the polygon to math the whole grid
    xmin = math.floor((xmin-x_ref)/bin_size)*bin_size+x_ref
    xmax = math.ceil((xmax-x_ref)/bin_size)*bin_size+x_ref
    ymin = math.floor((ymin-y_ref)/bin_size)*bin_size+y_ref
    ymax = math.ceil((ymax-y_ref)/bin_size)*bin_size+y_ref
    # Calculate the number of bins in each dimension
    xbins = int((xmax - xmin) / bin_size)
    ybins = int((ymax - ymin) / bin_size)
    # Create arrays of x and y coordinates for bins
    x_coords = np.arange(xmin, xmin + xbins * bin_size, bin_size)
    y_coords = np.arange(ymin, ymin + ybins * bin_size, bin_size)
    # Use NumPy's meshgrid to generate all combinations of x and y coordinates
    x_grid, y_grid = np.meshgrid(x_coords, y_coords)
    # Flatten the arrays and create bin polygons using vectorized operations
    x_flat = x_grid.flatten()
    y_flat = y_grid.flatten()
    bin_polygons = [
        Polygon([
            (x, y), (x + bin_size, y), (x + bin_size, y + bin_size), (x, y + bin_size)
        ]) for x, y in zip(x_flat, y_flat)
    ]
    return bin_polygons

In [77]:
polygon_df = gpd.read_file('Untitled Polygon 2.kml')
polygon_df = polygon_df.to_crs("EPSG:32648")
bins_polygons = [generate_bins(geom, 10) for geom in polygon_df['geometry']]
bins_polygons = [bin_poly for sublist in bins_polygons for bin_poly in sublist]
# Create the GeoDataFrame for bins
bins_df = gpd.GeoDataFrame({'geometry': bins_polygons}, crs=polygon_df.crs)
bins_df['centroid'] = bins_df['geometry'].centroid
bins_df_4326 = bins_df.to_crs("EPSG:4326")
bins_df_4326['centroid'] = bins_df_4326['centroid'].to_crs("EPSG:4326")
# bins_df_4326.to_file('bin2.kml', driver='KML')

In [78]:
bins_df_4326['longitude'] = bins_df_4326['centroid'].x
bins_df_4326['latitude'] = bins_df_4326['centroid'].y
bins_df_4326['bin_id'] = "bin" + "_" +bins_df_4326['latitude'].astype(str)+ "_" + bins_df_4326['longitude'].astype(str)
bins_df_4326 = bins_df_4326[['bin_id', 'geometry']]
bins_df_4326

,bin_id,geometry
0,bin_16.041217350723088_108.24072635661616,"POLYGON ((108.24068 16.04117, 108.24077 16.041..."
1,bin_16.041215938474608_108.24081969074888,"POLYGON ((108.24077 16.04117, 108.24087 16.041..."
2,bin_16.041214526185627_108.24091302487227,"POLYGON ((108.24087 16.04117, 108.24096 16.041..."
3,bin_16.041213113856124_108.24100635898637,"POLYGON ((108.24096 16.04117, 108.24105 16.041..."
4,bin_16.04121170148612_108.24109969309114,"POLYGON ((108.24105 16.04117, 108.24115 16.041..."
...,...,...
1595,bin_16.04395175690714_108.24497171721362,"POLYGON ((108.24492 16.04391, 108.24502 16.043..."
1596,bin_16.043950342576224_108.2450650522252,"POLYGON ((108.24502 16.04391, 108.24511 16.043..."
1597,bin_16.043948928204802_108.24515838722745,"POLYGON ((108.24511 16.04390, 108.24520 16.043..."
1598,bin_16.043947513792855_108.24525172222037,"POLYGON ((108.24520 16.04390, 108.24530 16.043..."
